In [1]:
from transformers import AutoModel, AutoTokenizer, EncoderDecoderModel, BertTokenizer, AdamW, BertModel, T5ForConditionalGeneration, T5Tokenizer
from datasets import load_dataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import math
import sys
import gc

In [18]:
model = T5ForConditionalGeneration.from_pretrained('t5-base')

In [19]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [21]:
model = model.to(device)

In [38]:
sentence = "To facilitate future work on transfer learning for NLP, we release our dataset, pre-trained models, and code."

text =  "paraphrase: " + sentence + " </s>"
#text = sentence + " </s>"

In [39]:
max_len = 256

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=120,
    top_p=0.98,
    early_stopping=True,
    num_return_sequences=10
)

In [40]:
print(f"Sentence: {sentence}")

print("Paraphrase: ")

for i,line in enumerate(beam_outputs):
    paraphrase = tokenizer.decode(line,skip_special_tokens=True,clean_up_tokenization_spaces=True)
    print(f"{i+1}. {paraphrase}")

Sentence: To facilitate future work on transfer learning for NLP, we release our dataset, pre-trained models, and code.
Paraphrase: 
1. False
2. True
3. False
4. True
5. False
6. True
7. False
8. True
9. True
10. False


In [2]:
raw = load_dataset('paws', 'labeled_final')

Reusing dataset paws (C:\Users\Pranav\.cache\huggingface\datasets\paws\labeled_final\1.1.0\09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34)


In [3]:
raw['train']

Dataset({
    features: ['id', 'sentence1', 'sentence2', 'label'],
    num_rows: 49401
})

In [4]:
model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-cased', 'bert-base-cased')

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.

In [5]:
from pympler import asizeof
asizeof.asizeof(model)

1845336

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [7]:
paraphrases = raw.filter(lambda example: example['label'] == 1)

Loading cached processed dataset at C:\Users\Pranav\.cache\huggingface\datasets\paws\labeled_final\1.1.0\09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34\cache-f8f4b05c63816056.arrow
Loading cached processed dataset at C:\Users\Pranav\.cache\huggingface\datasets\paws\labeled_final\1.1.0\09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34\cache-9bfc9aaee2aa1e33.arrow
Loading cached processed dataset at C:\Users\Pranav\.cache\huggingface\datasets\paws\labeled_final\1.1.0\09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34\cache-274702c64a43f78c.arrow


In [8]:
paraphrases

DatasetDict({
    train: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 21829
    })
    test: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 3536
    })
    validation: Dataset({
        features: ['id', 'sentence1', 'sentence2', 'label'],
        num_rows: 3539
    })
})

In [9]:
def tokenize(examples):
    return tokenizer(examples['sentence1'], padding='max_length', truncation=True)

In [10]:
tokenized_data = paraphrases.map(tokenize, batched=True)

Loading cached processed dataset at C:\Users\Pranav\.cache\huggingface\datasets\paws\labeled_final\1.1.0\09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34\cache-09d6db081c52014b.arrow
Loading cached processed dataset at C:\Users\Pranav\.cache\huggingface\datasets\paws\labeled_final\1.1.0\09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34\cache-070a8db3ae33f94e.arrow
Loading cached processed dataset at C:\Users\Pranav\.cache\huggingface\datasets\paws\labeled_final\1.1.0\09d8fae989bb569009a8f5b879ccf2924d3e5cd55bfe2e89e6dab1c0b50ecd34\cache-4145aa024d0eeff2.arrow


In [11]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.decoder.vocab_size

In [12]:
tokenized_data['train']

Dataset({
    features: ['attention_mask', 'id', 'input_ids', 'label', 'sentence1', 'sentence2', 'token_type_ids'],
    num_rows: 21829
})

In [13]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
device

device(type='cuda')

In [14]:
model.to(device)

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [15]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [16]:
len(paraphrases['train'])

21829

In [17]:
torch.cuda.memory_allocated(device=device)

981136896

In [111]:
train_loader = DataLoader(paraphrases['train'], shuffle=True, batch_size=32)
eval_loader = DataLoader(paraphrases['test'], shuffle=True, batch_size=3)

In [134]:
for batch in train_loader:
    print(gc.get_count())
#     print(gc.collect())
#     print(gc.get_count())
    print(torch.cuda.memory_allocated(device=device))
    input_ids = tokenizer(batch['sentence1'], return_tensors="pt", padding=True).input_ids.to(device)
    labels = tokenizer(batch['sentence2'], return_tensors="pt", padding=True).input_ids.to(device)
    outputs = model(input_ids=input_ids, decoder_input_ids=labels, labels=labels)
    print(torch.cuda.memory_allocated(device=device))
    del outputs
    #break

(528, 0, 0)
981158400
3922751488
(27, 1, 0)
981161472
3434172416
(45, 1, 0)
981157376
3956356608
(219, 1, 0)
981161472
3279288832
(434, 1, 0)
981156352
4008344576
(13, 2, 0)
981161984
3533673472
(31, 2, 0)
981158400
3896145408
(49, 2, 0)
981160960
3747096576
(70, 2, 0)
981159936
3591573504
(150, 2, 0)
981158912
4079945216
(366, 2, 0)
981162496
4873844224
(581, 2, 0)
981168640
3605023232
(13, 3, 0)
981158912
3524404224
(31, 3, 0)
981158400
3533673472
(49, 3, 0)
981158400
3673195008
(67, 3, 0)
981159424
4003525120
(85, 3, 0)
981161984
3774927872
(103, 3, 0)
981159936
3621320704
(134, 3, 0)
981159424
3875826688
(230, 3, 0)
981161472
4003525120
(448, 3, 0)
981161984
3747096576
(13, 4, 0)
981159936
3863938048
(46, 4, 0)
981160448
3750005760
(49, 4, 0)
981159424
3816769024
(65, 4, 0)
981160448
3691940864
(85, 4, 0)
981159424
3439453696
(103, 4, 0)
981157888
4234335232
(121, 4, 0)
981163520
3764068864
(139, 4, 0)
981160448
3816769024
(157, 4, 0)
981160448
3912532480
(236, 4, 0)
981161472
3533

(87, 11, 1)
981158912
3734717952
(102, 11, 1)
981159936
4165508608
(120, 11, 1)
981163008
3524404224
(138, 11, 1)
981158400
3488883712
(156, 11, 1)
981157888
3559145984
(174, 11, 1)
981158912
3331862528
(267, 11, 1)
981156864
3431250944
(481, 11, 1)
981157376
3822787072
(12, 0, 2)
981160448
3896395264
(30, 0, 2)
981160960
3315894784
(48, 0, 2)
981156864
3307445248
(66, 0, 2)
981156352
3559145984
(84, 0, 2)
981158912
4234335232
(102, 0, 2)
981163520
3549973504
(120, 0, 2)
981158400
3263878656
(138, 0, 2)
981156352
3549973504
(156, 0, 2)
981158400
3533673472
(174, 0, 2)
981158400
3887680000
(266, 0, 2)
981160960
3747096576
(498, 0, 2)
981159936
3717307904
(12, 1, 2)
981159424
3912532480
(30, 1, 2)
981161472
3737421824
(48, 1, 2)
981159936
3170318848
(66, 1, 2)
981155840
3332685312
(95, 1, 2)
981156864
3286192128
(102, 1, 2)
981156352
3434172416
(120, 1, 2)
981157376
3268554240
(138, 1, 2)
981155840
3605023232
(156, 1, 2)
981158912
4289705984
(174, 1, 2)
981164032
3482592768
(267, 1, 2)
9

3533673472
(69, 8, 3)
981158400
3922751488
(87, 8, 3)
981161472
3621320704
(117, 8, 3)
981159424
3922751488
(123, 8, 3)
981161472
3232141312
(141, 8, 3)
981155840
5559379456
(159, 8, 3)
981173248
3360667648
(177, 8, 3)
981157376
3747096576
(270, 8, 3)
981159936
3734717952
(485, 8, 3)
981159936
3482592768
(10, 9, 3)
981157376
4274568704
(30, 9, 3)
981164032
3519784448
(62, 9, 3)
981157888
3747096576
(66, 9, 3)
981159936
4363176448
(95, 9, 3)
981164544
3549973504
(102, 9, 3)
981158400
3653359616
(134, 9, 3)
981159424
3365527040
(138, 9, 3)
981156864
3747096576
(156, 9, 3)
981159936
3482592768
(174, 9, 3)
981157376
3506709504
(267, 9, 3)
981157888
4175022080
(481, 9, 3)
981163008
3747096576
(15, 10, 3)
981159936
3521523712
(33, 10, 3)
981158400
3482592768
(51, 10, 3)
981157376
3691940864
(69, 10, 3)
981159424
3331862528
(87, 10, 3)
981156864
3307445248
(108, 10, 3)
981156352
3586861056
(123, 10, 3)
981158912
3519784448
(141, 10, 3)
981157888
3521523712
(159, 10, 3)
981158400
4236253696
(1

In [127]:
del outputs

NameError: name 'outputs' is not defined

In [135]:
gc.get_objects()

[('reserved_bytes.small_pool.allocated', 715128832),
 ('reserved_bytes.small_pool.freed', 708837376),
 ('reserved_bytes.large_pool.current', 6899630080),
 ('reserved_bytes.large_pool.peak', 6975127552),
 ('reserved_bytes.large_pool.allocated', 18605932544),
 ('reserved_bytes.large_pool.freed', 11706302464),
 ('active_bytes.all.current', 981157376),
 ('active_bytes.all.peak', 6834945024),
 ('active_bytes.all.allocated', 5053816050176),
 ('active_bytes.all.freed', 5052834892800),
 ('active_bytes.small_pool.current', 1369600),
 ('active_bytes.small_pool.peak', 614649856),
 ('active_bytes.small_pool.allocated', 8721431552),
 ('active_bytes.small_pool.freed', 8720061952),
 ('active_bytes.large_pool.current', 979787776),
 ('active_bytes.large_pool.peak', 6831282688),
 ('active_bytes.large_pool.allocated', 5045094618624),
 ('active_bytes.large_pool.freed', 5044114830848),
 ('inactive_split_bytes.all.current', 103070208),
 ('inactive_split_bytes.all.peak', 1968448000),
 ('inactive_split_bytes.

In [129]:
gc.collect()

21

In [132]:
gc.get_count()

(171, 0, 0)

In [130]:
torch.cuda.memory_allocated(device=device)

981173248

In [65]:
torch.cuda.empty_cache()

In [ ]:
%load_ext line_profiler
epoch_bar = tqdm(range(10))
num_batches = math.ceil(len(paraphrases['train'])/512)

def train():
    for epoch in range(10):
        progress_bar = tqdm(range(num_batches))
        for batch in train_loader:
            input_ids = tokenizer(batch['sentence1'], return_tensors="pt", padding=True).input_ids.to(device)
            labels = tokenizer(batch['sentence2'], return_tensors="pt", padding=True).input_ids.to(device)
            outputs = model(input_ids=input_ids, decoder_input_ids=labels, labels=labels)
            optimizer.zero_grad()
            outputs.loss.backward()
            optimizer.step()
            del input_ids
            del labels
            del outputs
            progress_bar.update(1)

        model.eval()
        for batch in eval_loader:
            input_ids = tokenizer(batch['sentence1'], return_tensors="pt", padding=True).input_ids.to(device)
            labels = tokenizer(batch['sentence2'], return_tensors="pt", padding=True).input_ids.to(device)
            with torch.no_grad():
                generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder.start_token_id)
                print(tokenizer.decode(input_ids[0]))
                print(tokenizer.decode(generated[0]))
            break
        epoch_bar.update(1)

In [ ]:
%lprun -f train train()

In [ ]:
model.eval()
for batch in eval_loader:
    input_ids = tokenizer(batch['sentence1'], return_tensors="pt", padding=True).input_ids.to(device)
    with torch.no_grad():
        generated = model.generate(input_ids, decoder_start_token_id=model.config.decoder_start_token_id)
    print(tokenizer.decode(input_ids[0]))
    print(tokenizer.decode(generated[0]))
    break

In [ ]:
tok = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
enc = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
enc_in = tokenizer("What?!", return_tensors='pt')

In [ ]:
enc(**enc_in).last_hidden_state.shape